In [1]:
import os
import pandas as pd

In [2]:
path_now = os.getcwd()

In [3]:
path_docs = path_now + '\\..\\2.document_set'
path_train = path_now + '\\..\\3.training_data'
path_test = path_now + '\\..\\4.test_data'

In [4]:
path_docs

'C:\\Users\\Archit Umrao\\Desktop\\CSS Competiton\\Hackathon_Packet\\1.code\\..\\2.document_set'

### Loading the text data 

In [5]:
df= pd.read_csv(path_docs + '\\document_set.csv')

In [6]:
df.columns

Index(['Document_Id', 'Text'], dtype='object')

In [7]:
listofindex = df['Document_Id'].tolist()
doc_as_strings = df['Text'].tolist()

#### Loading train & test part 

In [8]:
df_train = pd.read_csv(path_train + '\\Training_Data.csv')
df_test = pd.read_csv(path_test + '\\Test_Data.csv')

In [9]:
df_train.columns, df_test.columns

(Index(['document_id', 'category'], dtype='object'),
 Index(['document_id', 'category'], dtype='object'))

In [10]:
docs_trained_on = df_train['document_id'].tolist()
docs_trained_on_cat = df_train['category'].tolist()
docs_to_test_on = df_test['document_id'].tolist()

In [11]:
len(docs_trained_on) + len(docs_to_test_on), len(listofindex)

(148665, 148665)

In [12]:
dict_train_id_category = dict(zip(docs_trained_on, docs_trained_on_cat))

## Preprocessing Corpus

In [13]:
import re
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [14]:
#Removing whitespaces
def whitespace_remover_and_tokeniser(list_of_docs):
    list_of_docs_whitespace_free = []
    for doc in list_of_docs:
        list_of_docs_whitespace_free.append(doc.split())
    return list_of_docs_whitespace_free

#Symbol Remover
def symbol_remover_and_lowercaser(list_of_docs):
    list_of_docs_symbol_free = []
    for doc in list_of_docs:
        temp_list = [] 
        for mixed_word in doc:
            word = re.split(r'\W+', mixed_word)
            if(len(word) == 1):
                temp_list.append(word[0].lower())
            elif(len(word) == 2):
                if(word[0] == r'\W'):
                    temp_list.append(word[1].lower())
                elif(word[1] == r'\W'):
                    temp_list.append(word[0].lower())
                elif(word[0] == r'\w' and word[1] == r'\w'):
                    temp_list.append(word[0].lower())
                    temp_list.append(word[1].lower())
        list_of_docs_symbol_free.append(temp_list)
    return list_of_docs_symbol_free

#Part of Speech Tagger: Taking too much time
def pos_tagger(list_of_docs):
    list_of_docs_pos_tagged = []
    for doc in list_of_docs:
        list_of_docs_pos_tagged.append(pos_tag(doc, tagset = 'universal'))
    return list_of_docs_pos_tagged

#Lemmatizer: convert into root lemma
def lemmatizer(list_of_docs):
    list_of_docs_lemmatized = []
    wnl = WordNetLemmatizer()
    for doc in list_of_docs:
        list_of_docs_lemmatized.append([ wnl.lemmatize(word) for word in doc] )
    return list_of_docs_lemmatized

#Remove stopwords: Taking too much time
def stopword_remover(list_of_docs):
    list_of_docs_stopword_cleaned = []
    for doc in list_of_docs:
        stopword_cleaned_doc = [word for word in doc if word not in stopwords.words('english')]
        list_of_docs_stopword_cleaned.append(stopword_cleaned_doc)
    return list_of_docs_stopword_cleaned
    

In [15]:
def main_preprocessing_function(list_of_docs):
    
    list_of_documents = whitespace_remover_and_tokeniser(list_of_docs)
    print("Whitespace removal and tokenising done")
    
    list_of_documents = symbol_remover_and_lowercaser(list_of_documents)
    print("Symbol removal and lowercasing done")
    
    #list_of_documents = pos_tagger(list_of_documents)
    #print("POS tagging done")
    
    list_of_documents = lemmatizer(list_of_documents)
    print("Lemmatization done")
    
    #list_of_documents = stopword_remover(list_of_documents)
    #print("Stopword removal done")
    
    return list_of_documents

In [16]:
list_of_preprocessed_docs = main_preprocessing_function(doc_as_strings)

Whitespace removal and tokenising done
Symbol removal and lowercasing done
Lemmatization done


In [17]:
dict_corpus = dict(zip(listofindex, list_of_preprocessed_docs))

# Training wor2vec on the entire corpus 

In [23]:
import gensim
import logging
import imp
imp.reload(gensim)

<module 'gensim' from 'C:\\Users\\Archit Umrao\\Anaconda3\\lib\\site-packages\\gensim\\__init__.py'>

In [24]:
gensim.__version__

'2.2.0'

In [26]:
import scipy 
scipy.__version__

'0.19.0'

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model = gensim.models.word2vec.Word2Vec(list_of_preprocessed_docs, min_count=1, window = 5, sample = 1e-1, size = 100)

2017-07-19 02:19:38,296 : WARNING : Slow version of gensim.models.word2vec is being used
2017-07-19 02:19:38,300 : INFO : collecting all words and their counts
2017-07-19 02:19:38,302 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-07-19 02:19:38,700 : INFO : PROGRESS: at sentence #10000, processed 781844 words, keeping 18636 word types
2017-07-19 02:19:39,021 : INFO : PROGRESS: at sentence #20000, processed 1620354 words, keeping 24341 word types
2017-07-19 02:19:39,360 : INFO : PROGRESS: at sentence #30000, processed 2414192 words, keeping 29519 word types
2017-07-19 02:19:39,726 : INFO : PROGRESS: at sentence #40000, processed 3257051 words, keeping 33692 word types
2017-07-19 02:19:40,077 : INFO : PROGRESS: at sentence #50000, processed 4090550 words, keeping 39747 word types
2017-07-19 02:19:40,359 : INFO : PROGRESS: at sentence #60000, processed 4647184 words, keeping 43219 word types
2017-07-19 02:19:40,685 : INFO : PROGRESS: at sentence #70000, p

2017-07-19 02:39:05,612 : INFO : PROGRESS: at 1.25% examples, 645 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:39:09,888 : INFO : PROGRESS: at 1.28% examples, 651 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:39:47,460 : INFO : PROGRESS: at 1.30% examples, 639 words/s, in_qsize 6, out_qsize 0
2017-07-19 02:39:48,791 : INFO : PROGRESS: at 1.31% examples, 646 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:39:53,016 : INFO : PROGRESS: at 1.34% examples, 652 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:40:29,760 : INFO : PROGRESS: at 1.35% examples, 641 words/s, in_qsize 6, out_qsize 0
2017-07-19 02:40:31,691 : INFO : PROGRESS: at 1.37% examples, 648 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:40:36,012 : INFO : PROGRESS: at 1.38% examples, 654 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:41:13,040 : INFO : PROGRESS: at 1.39% examples, 643 words/s, in_qsize 6, out_qsize 0
2017-07-19 02:41:14,981 : INFO : PROGRESS: at 1.41% examples, 650 words/s, in_qsize 6, out_qsize 0
2017-07-19

2017-07-19 02:59:08,143 : INFO : PROGRESS: at 2.58% examples, 666 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:59:10,612 : INFO : PROGRESS: at 2.59% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:59:17,506 : INFO : PROGRESS: at 2.61% examples, 672 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:59:54,216 : INFO : PROGRESS: at 2.63% examples, 666 words/s, in_qsize 5, out_qsize 0
2017-07-19 02:59:56,840 : INFO : PROGRESS: at 2.65% examples, 669 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:00:03,657 : INFO : PROGRESS: at 2.67% examples, 672 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:00:38,217 : INFO : PROGRESS: at 2.69% examples, 666 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:00:40,961 : INFO : PROGRESS: at 2.71% examples, 670 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:00:47,743 : INFO : PROGRESS: at 2.73% examples, 672 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:01:21,638 : INFO : PROGRESS: at 2.75% examples, 667 words/s, in_qsize 6, out_qsize 0
2017-07-19

2017-07-19 03:18:46,406 : INFO : PROGRESS: at 3.99% examples, 678 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:19:13,678 : INFO : PROGRESS: at 4.01% examples, 676 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:19:18,987 : INFO : PROGRESS: at 4.02% examples, 677 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:19:29,092 : INFO : PROGRESS: at 4.04% examples, 678 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:19:55,861 : INFO : PROGRESS: at 4.06% examples, 676 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:20:01,345 : INFO : PROGRESS: at 4.07% examples, 678 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:20:11,680 : INFO : PROGRESS: at 4.08% examples, 678 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:20:38,494 : INFO : PROGRESS: at 4.09% examples, 676 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:20:44,028 : INFO : PROGRESS: at 4.11% examples, 678 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:20:54,539 : INFO : PROGRESS: at 4.13% examples, 679 words/s, in_qsize 5, out_qsize 0
2017-07-19

2017-07-19 03:38:36,287 : INFO : PROGRESS: at 5.29% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:38:51,235 : INFO : PROGRESS: at 5.31% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:39:11,760 : INFO : PROGRESS: at 5.33% examples, 681 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:39:19,158 : INFO : PROGRESS: at 5.35% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:39:34,247 : INFO : PROGRESS: at 5.37% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:39:54,918 : INFO : PROGRESS: at 5.39% examples, 681 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:40:01,891 : INFO : PROGRESS: at 5.41% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 03:40:17,644 : INFO : PROGRESS: at 5.42% examples, 682 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:40:37,528 : INFO : PROGRESS: at 5.43% examples, 681 words/s, in_qsize 6, out_qsize 0
2017-07-19 03:40:44,454 : INFO : PROGRESS: at 5.44% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19

...

2017-07-19 22:10:21,529 : INFO : PROGRESS: at 74.25% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:10:53,247 : INFO : PROGRESS: at 74.26% examples, 682 words/s, in_qsize 6, out_qsize 0
2017-07-19 22:10:54,382 : INFO : PROGRESS: at 74.27% examples, 682 words/s, in_qsize 6, out_qsize 0
2017-07-19 22:11:07,235 : INFO : PROGRESS: at 74.28% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:11:39,636 : INFO : PROGRESS: at 74.30% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:11:40,754 : INFO : PROGRESS: at 74.31% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:11:52,834 : INFO : PROGRESS: at 74.32% examples, 682 words/s, in_qsize 6, out_qsize 0
2017-07-19 22:12:24,549 : INFO : PROGRESS: at 74.33% examples, 681 words/s, in_qsize 6, out_qsize 0
2017-07-19 22:12:26,513 : INFO : PROGRESS: at 74.35% examples, 682 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:12:37,570 : INFO : PROGRESS: at 74.36% examples, 682 words/s, in_qsize 5, out_qsize 0


2017-07-19 22:36:33,790 : INFO : PROGRESS: at 75.29% examples, 679 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:53:56,909 : INFO : PROGRESS: at 75.30% examples, 670 words/s, in_qsize 6, out_qsize 0
2017-07-19 22:54:02,601 : INFO : PROGRESS: at 75.31% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:54:11,911 : INFO : PROGRESS: at 75.32% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:54:58,129 : INFO : PROGRESS: at 75.34% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:55:00,806 : INFO : PROGRESS: at 75.36% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:55:09,414 : INFO : PROGRESS: at 75.38% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:55:47,016 : INFO : PROGRESS: at 75.39% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:55:49,394 : INFO : PROGRESS: at 75.40% examples, 670 words/s, in_qsize 5, out_qsize 0
2017-07-19 22:55:58,564 : INFO : PROGRESS: at 75.41% examples, 670 words/s, in_qsize 5, out_qsize 0


2017-07-19 23:16:50,916 : INFO : PROGRESS: at 76.20% examples, 668 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:16:52,073 : INFO : PROGRESS: at 76.21% examples, 668 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:17:00,103 : INFO : PROGRESS: at 76.23% examples, 668 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:17:45,282 : INFO : PROGRESS: at 76.24% examples, 668 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:17:54,512 : INFO : PROGRESS: at 76.26% examples, 668 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:18:40,295 : INFO : PROGRESS: at 76.27% examples, 668 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:18:49,572 : INFO : PROGRESS: at 76.29% examples, 668 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:19:33,989 : INFO : PROGRESS: at 76.30% examples, 668 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:19:42,518 : INFO : PROGRESS: at 76.33% examples, 668 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:20:24,756 : INFO : PROGRESS: at 76.34% examples, 668 words/s, in_qsize 6, out_qsize 0


2017-07-19 23:43:00,602 : INFO : PROGRESS: at 77.23% examples, 667 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:43:03,989 : INFO : PROGRESS: at 77.23% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:43:06,438 : INFO : PROGRESS: at 77.25% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:43:46,251 : INFO : PROGRESS: at 77.26% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:43:49,975 : INFO : PROGRESS: at 77.27% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:43:52,426 : INFO : PROGRESS: at 77.28% examples, 667 words/s, in_qsize 6, out_qsize 0
2017-07-19 23:44:32,971 : INFO : PROGRESS: at 77.29% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:44:36,114 : INFO : PROGRESS: at 77.30% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:44:38,102 : INFO : PROGRESS: at 77.31% examples, 667 words/s, in_qsize 5, out_qsize 0
2017-07-19 23:45:18,233 : INFO : PROGRESS: at 77.33% examples, 667 words/s, in_qsize 5, out_qsize 0


2017-07-20 00:10:54,366 : INFO : PROGRESS: at 78.24% examples, 665 words/s, in_qsize 6, out_qsize 0
2017-07-20 00:10:58,246 : INFO : PROGRESS: at 78.25% examples, 665 words/s, in_qsize 5, out_qsize 0
2017-07-20 00:11:37,319 : INFO : PROGRESS: at 78.26% examples, 664 words/s, in_qsize 5, out_qsize 0
2017-07-20 00:11:38,537 : INFO : PROGRESS: at 78.28% examples, 665 words/s, in_qsize 6, out_qsize 0
2017-07-20 00:11:42,445 : INFO : PROGRESS: at 78.29% examples, 665 words/s, in_qsize 5, out_qsize 0
2017-07-20 00:12:20,938 : INFO : PROGRESS: at 78.30% examples, 664 words/s, in_qsize 6, out_qsize 0
2017-07-20 00:12:25,803 : INFO : PROGRESS: at 78.32% examples, 665 words/s, in_qsize 6, out_qsize 0
2017-07-20 00:13:04,351 : INFO : PROGRESS: at 78.33% examples, 664 words/s, in_qsize 5, out_qsize 0
2017-07-20 00:13:09,212 : INFO : PROGRESS: at 78.35% examples, 665 words/s, in_qsize 6, out_qsize 0
2017-07-20 00:13:47,414 : INFO : PROGRESS: at 78.35% examples, 665 words/s, in_qsize 5, out_qsize 0


## Using Doc2vec, a single vector representing 1 document

In [ ]:
def doc2vec_calculator(list_of_preprocessed_docs, w2v_model):
    doc2vec_list = []
    for doc in list_of_preprocessed_docs:
        sum_of_weights = 0
        sum_of_numerical_vectors = np.zeros((100,))
        for word in doc:
            word_vector = w2v_model.wv[word]  
            sum_of_numerical_vectors+= word_vector
            sum_of_weights+= 1
        doc2vec_list.append((sum_of_numerical_vectors / sum_of_weights))
    return doc2vec_list

In [ ]:
doc2vec_list = doc2vec_calculator(list_of_preprocessed_docs, w2v_model)

## KNN mapping on data

In [ ]:
from sklearn import neighbors

In [ ]:
dict_train={}
for index in docs_trained_on[ :int(len(docs_trained_on)*0.8)]:
    dict_train.update( {index : dict_corpus[index] } )
dict_cv={}
for index in docs_trained_on[int(len(docs_trained_on)*0.8): ]:
    dict_cv.update( {index : dict_corpus[index] } )

In [ ]:
#Unzipping dictionary
indexes_train , doc2vec_train = zip(*dict_train.items())
indexes_cv , doc2vec_cv = zip(*dict_cv.items())

In [ ]:
knn_object = neighbors.KNeighborsClassifier( n_neighbors=5, leaf_size= 35)
knn_object.fit(doc2vec_train, indexes_train)

predicted = knn_object.predict(doc2vec_cv)
actual = indexes_cv

correct = 0 
wrong = 0
for i in range( len(numpy_num_features[2700:]) ):
    if (predicted[i] == actual[i]):
        correct+= 1
    else:
        wrong+= 1

print("Accuracy is {} ".format(correct/(correct + wrong)))